In [18]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jun 11 11:02:03 2021

@author: Christian
"""

from mip import Model, xsum, maximize, BINARY

from Co2_price import df_data
from electricity_demand import electricity_demand
from Electricity_Price0 import Electricity_price_pivot0
from Electricity_Price1 import Electricity_price_pivot1
from gas_price import gas_p
from heat_demand import heat_demand_pivot
from heat_price import heat_p
from elec_effciency import elec_eff
from electricity_capacity import elec_capacity
from heat_capacity import heat_capacity


In [12]:


elec_demand = electricity_demand.stack().tolist()
elec_price_wholesale = Electricity_price_pivot0
elec_price_household_32_05 = Electricity_price_pivot1
co2_price = df_data
gas_price = gas_p
heat_demand_1 = heat_demand_pivot
heat_price_1 = heat_p


el_price = elec_price_wholesale.stack().tolist() # Electricity price
del_t = .25 # duration of time step
el_demand = electricity_demand.stack().tolist() # Electicity demand


gas_pp = gas_price["Preis"].values.tolist() # Gas price for different plants
em_fc = 0.2 # emission factor
co2_p = co2_price["mean_CO2_tax"].values.tolist() # CO2 Price
capacity_el = elec_capacity.stack().tolist() # maximum capacity of electricity for the power plants
capacity_ht = heat_capacity.stack().tolist() # maximum capacity of heat for the power plants
heat_demand = heat_demand_1.stack().tolist() # heat demand
heat_price = heat_price_1.stack().tolist() # heat price
heat_ratio = 400/385 # heat ratio


eff_plants = elec_eff.stack().tolist() # the efficiency of the power plants


T = range(len(el_price))
I = range(len(gas_pp))

m = Model("Maximizing profit", sense=maximize)
 

#variable
y_t = [[m.add_var(lb = 0) for i in I] for t in T] # Electricity generation
el_sold = [[m.add_var(lb = 0) for i in I] for t in T] # electricity sold 
# el_sold = [m.add_var(lb = 0) for t in T] # electricity sold 
x_t = [[m.add_var(lb = 0) for i in I] for t in T] # Fuel consumption
z_t = [[m.add_var(lb = 0) for i in I] for t in T] # Heat generation
# objective function

# Revenue = Electricity price * demand/generation
# Cost = Fuel price = gas price + emission_factor * co2 price

# Max Proft = Revenue - Cost



In [35]:
heat_demand_norm = [((element/max(heat_demand))*385) for element in heat_demand] 
heat_demand_norm[:10]

[60.070677643658726,
 75.91105388403895,
 73.66910599823679,
 80.2775568238312,
 99.11655431158485,
 134.28463950258848,
 145.66153611602735,
 137.80387243886756,
 136.23438131790192,
 129.5761661396915]

In [24]:
# m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)


In [17]:
m.objective = xsum((el_price[t] * del_t * el_sold[t][i]) - (x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand_norm[t][i] * heat_price[t][i]) for t in T for i in I)


KeyError: 0

In [2]:
m.objective = xsum((el_price[t] * del_t * el_sold[t]) - ( x_t[t][i] * ( gas_p[i] * del_t + (em_fc * co2_p[t]*del_t))) for t in T for i in I) + ((heat_demand[t] * heat_price[t]) for t in T)

# constraints

for t in T:
    m += xsum(y_t[t][i] for i in I) == el_demand[t]
    
    for i in I:
        m += y_t[t][i] <= capacity_el[i] # electricity generation <= maximum capacity of electricity of the plant

        m += z_t[t][i] <= capacity_ht[i] # heat generation <= maximum capacity of heat of the plant 
        
        m += x_t[t][i] == y_t[t][i]/eff_plants[i] # fuel consumption = Electricity generation / efficiency of the plants 

        m += heat_demand[t] <= z_t[t][i] # heat demand >= Heat generation  

        m += y_t[t][i] == heat_ratio * z_t[t][i] # electricity generation >= heat ratio * heat generation

        m += y_t[t][i] == el_sold[t] + el_demand[t]



status = m.optimize()
obj= m.objective_value

41.709999999999994

In [3]:
status

<OptimizationStatus.OPTIMAL: 0>

In [7]:
r = [3, 5, 8, 3, 6, 5, 7, 3]


In [8]:
type(r)

list

In [6]:
from electricity_demand import electricity_demand
import numpy as np
x = electricity_demand.stack().tolist()
x

[63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 63.974999999999994,
 54.150000000000006,
 49.075,
 48.625,
 56.7,
 65.4,
 75.4,
 103.625,
 168.375,
 225.85,
 231.02499999999998,
 238.5,
 228.02499999999998,
 191.8,
 172.05,
 185.2,
 199.95,
 204.275,
 182.175,
 126.87500000000001,
 98.85,
 86.85,
 77.25,
 72.8,
 71.075,
 75.44999999999999,
 66.27499999999999,
 63.625,
 65.35,
 69.275,
 76.275,
 105.325,
 158.375,
 190.37500000000003,
 201.24999999999997,
 207.875,
 200.975,
 180.4,
 140.875,
 101.475,
 85.025,

In [7]:
len(x)

8784